# Analyze prior vs posterior

In [ ]:
import subprocess
import chaospy
import numpy as np
import os, json
import scipy.interpolate as interp
from sklearn.cross_decomposition import PLSRegression
import scipy.optimize
import matplotlib.pyplot as plt

import matplotlib.patches as mpatches

def read_json(jsonfilename):
    with open(jsonfilename, 'r') as j:
        jsondata = json.loads(j.read())
        
    return jsondata

def resample(base_date, custom_date, data):
    
    f = interp.interp1d(custom_date, data, fill_value='extrapolate')
    new_data = f(base_date)
    return new_data

def save_summary(studyname, method, it):

    study_path = f"./simulations/studies/IE_SPE10.json"
    study = read_json(study_path)

    ensemble_storage = study['simulation']['storage']

    # get porosity

    permx = []
    for casename, path in study['extraction']['static3d'].items():

        path = study['extraction']['static3d'][casename]['PERMX']
        p = np.load(path)
        permx.append(p)

    permx = np.array(permx)

    fakehistory_path = f'./data/SPE10/spe10model2/historic_data/'
    permx_hist = np.load(os.path.join(fakehistory_path, 'static3d', 'PERMX.npy'))

    prior_study_path = './simulations/studies/IE_SPE10.json'
    post_study_path = './simulations/studies/HM_SPE10.json'

    prior_study = read_json(prior_study_path)
    post_study = read_json(post_study_path)

    history_kws = ['WWIR:I1', 'WWIR:I2', 'WWIR:I3', 
                'WWCT:P1', 'WWCT:P2', 'WWCT:P3', 'WWCT:P4',
                'WOPR:P1', 'WOPR:P2', 'WOPR:P3', 'WOPR:P4']

    history_years = np.load(os.path.join(fakehistory_path, 'summary', 'YEARS.npy'))

    real_folder = f"./figs/npys/{studyname}/{it}"
    os.makedirs(real_folder, exist_ok=True)   
    
    summary_dict = dict()
    summary_hist = dict()
    # get error terms
    for kw in history_kws:
        summary_dict[kw] = {}
        
        sum_history = np.load(os.path.join(fakehistory_path, 'summary', kw + '.npy'))
        first_realname = list(study['extraction']['summary'].keys())[0] 
        base_years = np.load(study['extraction']['summary'][first_realname]['YEARS'])
        sum_history = resample(base_years, history_years, sum_history)
        summary_hist[kw] = sum_history
        
        # load prior
        summary_dict[kw]['prior'] = []
        for casename, path in prior_study['extraction']['summary'].items():
            sum_simulation = np.load(prior_study['extraction']['summary'][casename][kw])
            sum_years = np.load(prior_study['extraction']['summary'][casename]['YEARS'])
            sum_simulation = resample(base_years, sum_years, sum_simulation)
            summary_dict[kw]['prior'].append(sum_simulation)
        
        # load posterior
        summary_dict[kw]['post'] = []
        for casename, path in post_study['extraction']['summary'].items():
            sum_simulation = np.load(post_study['extraction']['summary'][casename][kw])
            sum_years = np.load(post_study['extraction']['summary'][casename]['YEARS'])
            sum_simulation = resample(base_years, sum_years, sum_simulation)
            summary_dict[kw]['post'].append(sum_simulation)
            
    # fig, ax = plt.subplots(len(history_kws),1,figsize=(10,3*len(history_kws)))

    for i, kw in enumerate(history_kws):        
        np.save(f"{real_folder}/summarypost_{kw}.npy", summary_dict[kw]['post'])
        np.save(f"{real_folder}/summaryprior_{kw}.npy", summary_dict[kw]['prior'])
    np.save(f"{real_folder}/summarypost_YEARS.npy", base_years)

In [ ]:
import statistics
import matplotlib.colors

def save_average_prop(studyname, method, it):
    nx = 60
    ny = 60
    nz = 1
    
    prior_study_path = './simulations/studies/IE_SPE10.json'
    post_study_path = './simulations/studies/HM_SPE10.json'

    prior_study = read_json(prior_study_path)
    post_study = read_json(post_study_path)
    
    permx = []
    realizations = prior_study['extraction']['static3d']
    for real in realizations.keys():
        permx.append(np.load(realizations[real]['PERMX']))
    permx = np.array(permx)

    permx_post = []
    realizations = post_study['extraction']['static3d']
    for real in realizations.keys():
        permx_post.append(np.load(realizations[real]['PERMX']))
    permx_post = np.array(permx_post)

    fakehistory_path = f'./data/SPE10/spe10model2/historic_data/'
    permx_hist = np.load(os.path.join(fakehistory_path, 'static3d', 'PERMX.npy'))
    
    real_folder = f"./figs/npys/{studyname}/{it}"
    os.makedirs(real_folder, exist_ok=True)   

    # fig, ax = plt.subplots(1, 3, figsize=(15,5))
    for i in range(nz):
        for j, prop in enumerate([permx_hist, permx_post, permx]):
            if j in [1,2]:
                prop3d = np.exp(np.mean(np.log(prop), axis=0)).reshape((nx, ny, nz), order='F')
            else:
                prop3d = prop.reshape((nx, ny, nz), order='F')
        
            if j == 0:
                np.save(f"{real_folder}/2dpropHist.npy", prop3d[:, :, i])
            elif j == 1:
                np.save(f"{real_folder}/2dpropPost.npy", prop3d[:, :, i])
            elif j == 2:
                np.save(f"{real_folder}/2dpropPrior.npy", prop3d[:, :, i])

    # plt.savefig(f"./figs/npys/{studyname}/spe10_prop_average_{method}.png")

In [ ]:
def save_realizations(studyname, method, it:int):
    nx = 60
    ny = 60
    nz = 1
    
    post_study_path = './simulations/studies/HM_SPE10.json'
    post_study = read_json(post_study_path)
    
    permx_post = []
    realizations = post_study['extraction']['static3d']
    for real in realizations.keys():
        permx_post.append(np.load(realizations[real]['PERMX']))
    permx_post = np.array(permx_post)
    
    real_folder = f"./figs/npys/{studyname}/{it}"
    os.makedirs(real_folder, exist_ok=True)   
    
    for k in range(nz):
        prop2ds = []
        for i in range(4):
            for j in range(8):
            
                index = i*8 + j
                prop = permx_post[index,:]
                prop3d = prop.reshape((nx, ny, nz), order='F')
                # prop2d = np.mean(prop3d, axis=2)
                prop2d = prop3d[:, :, k]
                prop2ds.append(prop2d)

        for i in range(4):
            for j in range(8):
                index = i*8 + j
                prop = prop2ds[index]
                np.save(f"{real_folder}/2dprop_{index}.npy", prop)
        
    
    


In [ ]:
def save_prior(studyname, method, it):
    nx = 60
    ny = 60
    nz = 1
    
    prior_study_path = './simulations/studies/IE_SPE10.json'

    prior_study = read_json(prior_study_path)
    
    permx = []
    realizations = prior_study['extraction']['static3d']
    for real in realizations.keys():
        permx.append(np.load(realizations[real]['PERMX']))
    permx = np.array(permx)
    
    real_folder = f"./figs/npys/{studyname}/{it}"
    os.makedirs(real_folder, exist_ok=True)  
    
    for k in range(nz):

        prop2ds = []
        for i in range(4):
            for j in range(8):
            
                index = i*8 + j
                prop = permx[index,:]
                prop3d = prop.reshape((nx, ny, nz), order='F')
                # prop2d = np.mean(prop3d, axis=2)
                prop2d = prop3d[:, :, k]
                prop2ds.append(prop2d)
                
        for i in range(4):
            for j in range(8):
                index = i*8 + j
                prop = prop2ds[index]
                np.save(f"{real_folder}/prior_2dprop_{index}.npy", prop)

In [ ]:
import os
import subprocess

for i in range(1, 2):
    ## IF WE WANT TO RUN MORE THAN ONE INITIAL ENSEMBLE
    # subprocess.run(["python3", "./src/create_ensemble.py", f"./data/SPE10/spe10model2/SPE10_PG8607.json"])
    # subprocess.run(["python3", "./src/run_ensemble.py", "/usr/bin/flow", f"./simulations/studies/IE_SPE10.json"])
    # subprocess.run(["python3", "./src/extract_ensemble.py", f"./simulations/studies/IE_SPE10.json"])

    
    pairs = [
                (f"III_PLSR_4_ITS_PO_1_NC_3_{i}", "PLSR", [3, 3, 3, 3], [1, 1, 1, 1]),
                # (f"III_PLSR_4_ITS_PO_1_NC_4_{i}", "PLSR", [4, 4, 4, 4], [1, 1, 1, 1]),
                # (f"III_ESMDA_4_ITS_NC_3_{i}", "ESMDA", [3, 3, 3, 3], [4.0, 4.0, 4.0, 4.0]),
                # (f"III_ESMDA_4_ITS_NC_4_{i}", "ESMDA", [4, 4, 4, 4], [4.0, 4.0, 4.0, 4.0]),
            ]
    
    # pairs = [
    #             (f"II_PLSR_2_ITS_PO_1_NC_3_{i}", "PLSR", [3, 3], [1, 1]),
    #             (f"II_PLSR_2_ITS_PO_1_NC_4_{i}", "PLSR", [4, 4], [1, 1]),
    #             (f"II_PLSR_2_ITS_PO_2_NC_3_{i}", "PLSR", [3, 3], [2, 2]),
    #             (f"II_PLSR_2_ITS_PO_2_NC_4_{i}", "PLSR", [4, 4], [2, 2]),
    #             (f"II_ESMDA_2_ITS_NC_3_{i}", "ESMDA", [3, 3], [2.0, 2.0]),
    #             (f"II_ESMDA_2_ITS_NC_4_{i}", "ESMDA", [4, 4], [2.0, 2.0]),
    #         ]

    # pairs = [
    #         (f"I_PCA_2_ITS_PO_1_NC_2_{i}", "PCA", [2, 2], [1, 1]),
    #         (f"I_PCA_2_ITS_PO_2_NC_2_{i}", "PCA", [2, 2], [2, 2]),
    #         (f"I_PCA_2_ITS_PO_3_NC_2_{i}", "PCA", [2, 2], [3, 3]),
    #         (f"I_FICA_2_ITS_PO_1_NC_2_{i}", "FICA", [2, 2], [1, 1]),
    #         (f"I_FICA_2_ITS_PO_2_NC_2_{i}", "FICA", [2, 2], [2, 2]),
    #         (f"I_FICA_2_ITS_PO_3_NC_2_{i}", "FICA", [2, 2], [3, 3]),
    #         (f"I_PLSR_2_ITS_PO_1_NC_2_{i}", "PLSR", [2, 2], [1, 1]),
    #         (f"I_PLSR_2_ITS_PO_2_NC_2_{i}", "PLSR", [2, 2], [2, 2]),
    #         (f"I_PLSR_2_ITS_PO_3_NC_2_{i}", "PLSR", [2, 2], [3, 3]),
    #         (f"I_PCA_2_ITS_PO_1_NC_3_{i}", "PCA", [3, 3], [1, 1]),
    #         (f"I_PCA_2_ITS_PO_2_NC_3_{i}", "PCA", [3, 3], [2, 2]),
    #         (f"I_PCA_2_ITS_PO_3_NC_3_{i}", "PCA", [3, 3], [3, 3]),
    #         (f"I_FICA_2_ITS_PO_1_NC_3_{i}", "FICA", [3, 3], [1, 1]),
    #         (f"I_FICA_2_ITS_PO_2_NC_3_{i}", "FICA", [3, 3], [2, 2]),
    #         (f"I_FICA_2_ITS_PO_3_NC_3_{i}", "FICA", [3, 3], [3, 3]),
    #         (f"I_PLSR_2_ITS_PO_1_NC_3_{i}", "PLSR", [3, 3], [1, 1]),
    #         (f"I_PLSR_2_ITS_PO_2_NC_3_{i}", "PLSR", [3, 3], [2, 2]),
    #         (f"I_PLSR_2_ITS_PO_3_NC_3_{i}", "PLSR", [3, 3], [3, 3]),
    #         (f"I_PCA_2_ITS_PO_1_NC_4_{i}", "PCA", [4, 4], [1, 1]),
    #         (f"I_PCA_2_ITS_PO_2_NC_4_{i}", "PCA", [4, 4], [2, 2]),
    #         (f"I_PCA_2_ITS_PO_3_NC_4_{i}", "PCA", [4, 4], [3, 3]),
    #         (f"I_FICA_2_ITS_PO_1_NC_4_{i}", "FICA", [4, 4], [1, 1]),
    #         (f"I_FICA_2_ITS_PO_2_NC_4_{i}", "FICA", [4, 4], [2, 2]),
    #         (f"I_FICA_2_ITS_PO_3_NC_4_{i}", "FICA", [4, 4], [3, 3]),
    #         (f"I_PLSR_2_ITS_PO_1_NC_4_{i}", "PLSR", [4, 4], [1, 1]),
    #         (f"I_PLSR_2_ITS_PO_2_NC_4_{i}", "PLSR", [4, 4], [2, 2]),
    #         (f"I_PLSR_2_ITS_PO_3_NC_4_{i}", "PLSR", [4, 4], [3, 3]),
    #     ]
        
    for studyname, method, n_components, hyperparameters in pairs:
        print(studyname)
        paths = f"./figs/npys/{studyname}"
        if os.path.exists(paths):
            pass
        else:
            os.mkdir(paths)

        
        save_prior(studyname, method, 0)
        
        subprocess.run(["python3", "./src/hm_ensemble.py", "./simulations/studies/IE_SPE10.json", method, str(n_components[0]), str(hyperparameters[0])])
        subprocess.run(["python3", "./src/create_ensemble.py", "./data/SPE10/spe10model2/SPE10_PG8607_POST.json"])
        subprocess.run(["python3", "./src/run_ensemble.py", "/usr/bin/flow", "./simulations/studies/HM_SPE10.json"])
        subprocess.run(["python3", "./src/extract_ensemble.py", "./simulations/studies/HM_SPE10.json"])
        
        save_summary(studyname, method, 1)
        save_average_prop(studyname, method, 1)
        save_realizations(studyname, method, 1)

        for j, (n_component, hyperparameter) in enumerate(zip(n_components[1:], hyperparameters[1:])):
            subprocess.run(["python3", "./src/hm_ensemble.py", "./simulations/studies/HM_SPE10.json", method, str(n_component), str(hyperparameter)])
            subprocess.run(["python3", "./src/create_ensemble.py", "./data/SPE10/spe10model2/SPE10_PG8607_POST.json"])
            subprocess.run(["python3", "./src/run_ensemble.py", "/usr/bin/flow", "./simulations/studies/HM_SPE10.json"])
            subprocess.run(["python3", "./src/extract_ensemble.py", "./simulations/studies/HM_SPE10.json"])
            
            save_summary(studyname, method, j+2)
            save_average_prop(studyname, method, j+2)
            save_realizations(studyname, method, j+2)
            
        
    